In [1]:
import pandas as pd

In [2]:
atlantic = pd.read_csv('/Users/danielbrechner/hurrdat_co2/assets/atlantic.csv')
atlantic.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,HU,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,HU,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,HU,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,HU,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [12]:
atlantic.shape

(49105, 18)

In [13]:
atlantic.describe()

,Date,Maximum Wind,Low Wind NE,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
count,4.910500e+04,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000,49105.000000
mean,1.949802e+07,52.005091,-868.671031,-869.315772,-872.676428,-871.409245,-875.570940,-875.765319,-876.682028,-876.323205,-877.564321,-877.655045,-877.923552,-877.786295
std,4.461850e+05,27.681902,353.299666,351.548910,342.153768,345.628984,333.651396,333.133376,330.526085,331.535176,328.027275,327.774794,327.021430,327.408253
min,1.851062e+07,-99.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
25%,1.911110e+07,35.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
50%,1.956093e+07,45.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
75%,1.989081e+07,70.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
max,2.015111e+07,165.000000,710.000000,600.000000,640.000000,530.000000,360.000000,300.000000,330.000000,360.000000,180.000000,250.000000,150.000000,180.000000


In [10]:
atlantic.isnull().values.any() 

False

Dropping unnecessary columns

In [3]:
atlantic.drop(['ID', 'Time', 'Minimum Pressure', 'Event'], axis = 1, inplace= True)

The naming converntion for hurricanes started in 1950. This also might indicate better data quality, so we will use hurricane data from 1950 to present day and sort hurricanes by name

In [4]:
named =[]

for date in atlantic['Date']:
    if date > 19500000:
        named.append(True)
    else: 
        named.append(False)
date_range = pd.Series(named)
atlantic_df = atlantic[date_range]

atlantic_df.head()

,Name,Date,Status,Latitude,Longitude,Maximum Wind,Low Wind NE,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
21879,ABLE,19500812,TS,17.1N,55.5W,35,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
21880,ABLE,19500812,TS,17.7N,56.3W,40,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
21881,ABLE,19500812,TS,18.2N,57.4W,45,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
21882,ABLE,19500812,TS,19.0N,58.6W,50,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
21883,ABLE,19500813,TS,20.0N,60.0W,50,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


Dropping duplicate storms

In [6]:
atlantic_df = atlantic_df.drop_duplicates(subset='Name', keep="first")
atlantic_df

,Name,Date,Status,Latitude,Longitude,Maximum Wind,Low Wind NE,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
21879,ABLE,19500812,TS,17.1N,55.5W,35,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
21930,BAKER,19500818,TD,12.0N,54.0W,30,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
21990,CHARLIE,19500821,TS,11.1N,26.4W,35,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
22051,DOG,19500830,HU,15.2N,55.6W,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
22125,EASY,19500901,TS,19.7N,83.2W,40,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48170,DORIAN,20130722,LO,11.3N,20.0W,25,0,0,0,0,0,0,0,0,0,0,0,0
48241,FERNAND,20130825,TD,18.7N,94.2W,30,0,0,0,0,0,0,0,0,0,0,0,0
48690,GONZALO,20141011,LO,16.4N,54.9W,25,0,0,0,0,0,0,0,0,0,0,0,0
48939,NINE,20150915,LO,11.5N,40.3W,20,0,0,0,0,0,0,0,0,0,0,0,0
